In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
os.getcwd()

'/Users/caseyh/Desktop/footy/data_import'

In [3]:
main=pd.read_csv('/Users/caseyh/Desktop/footy/player_stats.csv')

In [4]:
main.head()

,Player,round,team,opponent,Disposals,Kicks,Handballs,Goals,Behinds,Marks,...,Goal Assists,Frees,Frees Against,Inside 50s,Rebounds,% Played,Subs,Clangers,One Percenters,Bounces
0,"Bowey, Jake",R1,MEL,WBD,17.0,10.0,7.0,0.0,0.0,3.0,...,0.0,1.0,0.0,3.0,3.0,73.0,-,0.0,1.0,0.0
1,"Brayshaw, Angus",R1,MEL,WBD,22.0,14.0,8.0,0.0,0.0,10.0,...,0.0,1.0,1.0,2.0,4.0,86.0,-,3.0,0.0,0.0
2,"Brown, Ben",R1,MEL,WBD,15.0,12.0,3.0,4.0,1.0,7.0,...,1.0,1.0,0.0,2.0,1.0,81.0,-,2.0,1.0,0.0
3,"Chandler, Kade",R1,MEL,WBD,16.0,11.0,5.0,1.0,2.0,8.0,...,2.0,1.0,0.0,4.0,0.0,81.0,-,1.0,0.0,0.0
4,"Fritsch, Bayley",R1,MEL,WBD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def rearrange(value):
    return '-'.join(' '.join(value.lower().split(', ')[::-1]).split(' '))

In [7]:
main['Player'] = main['Player'].apply(lambda x: rearrange(x))

In [8]:
mapping = {'CAR':'carlton-blues','MEL':'melbourne-demons','BRL':'brisbane-lions','STK':'st-kilda-saints','COL':'collingwood-magpies','GCS':'gold-coast-suns','GWS':'greater-western-sydney-giants','FRE':'fremantle-dockers','POR':'port-adelaide-power','WCE':'west-coast-eagles','SYD':'sydney-swans','NTH':'kangaroos','HAW':'hawthorn-hawks','GEE':'geelong-cats','RCH':'richmond-tigers','WBD':'western-bulldogs','ESS':'essendon-bombers','ADE':'adelaide-crows'}

In [9]:
for key in mapping.keys():
    main.loc[main['team']==key, 'team'] = mapping[key]

In [10]:
dic = main.groupby(['Player', 'team']).count().reset_index()[['Player', 'team']].to_dict()

In [ ]:
%time
output = pd.DataFrame({}, columns = ['Player','position','age','games played','dob','origin','height','weight'])
for i in range(len(dic['Player'])):
    try:
        url = f"https://www.footywire.com/afl/footy/pp-{dic['team'][i]}--{dic['Player'][i]}" 
    
        soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        info1 = soup.find_all(id = 'playerProfileData1')[0].text.split()
        info2 = soup.find_all(id = 'playerProfileData2')[0].text.split()

        info = info1 + info2

        lst_index = []
        for j, value in enumerate(info):
            if value in ['Age:', 'Games:', 'Born:', 'Origin:', 'Height:', 'Weight:','Position:']:
                lst_index.append(j)

        info_list = []
        for k in range(len(lst_index) - 1):
            a = lst_index[k]
            b = lst_index[k+1]
            info_list.append(info[a + 1:b])
        info_list.append(info[lst_index[-1] + 1:])

        print(i)
        print(' '.join(dic['Player'][i].split('-')))
        print(info_list)

        age = ' '.join(info_list[0])
        games = ' '.join(info_list[1])
        birth = ' '.join(info_list[2])
        place = ' '.join(info_list[3])
        height = ' '.join(info_list[4])
        weight = ' '.join(info_list[5])
        position = ' '.join(info_list[6])



        new_row = {'Player': ' '.join(dic['Player'][i].split('-')), 'position': position, 'age': age, 'games played':games, 
              'dob': birth, 'origin': place, 'height':height, 'weight':weight}

        output.loc[len(output)] = new_row
    except:
        continue


CPU times: user 7 µs, sys: 31 µs, total: 38 µs
Wall time: 14.1 µs
0
aaron cadman
[['19yr', '3mth'], ['9'], ['March', '3,', '2004'], ['Greater', 'Western', 'Victoria', 'Rebels'], ['195cm'], ['92kg'], ['Forward']]
1
aaron francis
[['25yr', '10mth'], ['63'], ['August', '10,', '1997'], ['West', 'Adelaide'], ['193cm'], ['92kg'], ['Defender,', 'Forward']]
2
aaron hall
[['32yr', '7mth'], ['161'], ['November', '9,', '1990'], ['Hobart'], ['185cm'], ['85kg'], ['Defender']]
3
aaron naughton
[['23yr', '6mth'], ['114'], ['November', '30,', '1999'], ['Peel', 'Thunder'], ['195cm'], ['93kg'], ['Forward']]
4
adam cerra
[['23yr', '8mth'], ['108'], ['October', '7,', '1999'], ['Eastern', 'Ranges'], ['187cm'], ['86kg'], ['Midfield']]
5
adam kennedy
[['30yr', '11mth'], ['153'], ['July', '12,', '1992'], ['Western', 'Jets'], ['184cm'], ['80kg'], ['Defender,', 'Forward']]
6
adam saad
[['28yr', '11mth'], ['165'], ['July', '23,', '1994'], ['Coburg'], ['178cm'], ['77kg'], ['Defender']]
7
adam tomlinson
[['29yr', 

65
brandon ellis
[['29yr', '10mth'], ['238'], ['August', '3,', '1993'], ['Calder', 'Cannons'], ['181cm'], ['82kg'], ['Defender,', 'Midfield']]
66
brandon starcevich
[['23yr', '11mth'], ['85'], ['July', '24,', '1999'], ['East', 'Perth'], ['187cm'], ['92kg'], ['Defender']]
67
brandon walker
[['20yr', '8mth'], ['42'], ['October', '17,', '2002'], ['East', 'Fremantle'], ['186cm'], ['80kg'], ['Defender']]
68
brandon zerk thatcher
[['24yr', '10mth'], ['42'], ['August', '25,', '1998'], ['Sturt'], ['195cm'], ['89kg'], ['Defender']]
69
brayden fiorini
[['25yr', '10mth'], ['88'], ['August', '22,', '1997'], ['Northern', 'Knights'], ['186cm'], ['84kg'], ['Midfield']]
70
brayden maynard
[['26yr', '9mth'], ['175'], ['September', '20,', '1996'], ['Sandringham', 'Dragons'], ['189cm'], ['93kg'], ['Defender']]
71
brennan cox
[['24yr', '10mth'], ['96'], ['August', '13,', '1998'], ['Woodville-West', 'Torrens'], ['195cm'], ['100kg'], ['Defender']]
72
brent daniels
[['24yr', '3mth'], ['73'], ['March', '9,', 

In [ ]:
output.to_csv('/Users/caseyh/Desktop/footy/playerinfo.csv')